In [ ]:
## VIEW 테이블 생성 ##
# 1. jentestore_trends, marieclaire_trends, vogue_trends, wkorea_trends, wwdkorea_trends 테이블을 하나의 VIEW 테이블로 생성

import pymysql

# MySQL 연결
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='0805',
    database='fashion_trends',
    charset='utf8mb4'
)
cursor = conn.cursor()

# VIEW 생성 SQL
create_view_sql = """
CREATE OR REPLACE VIEW all_trends AS
SELECT 'jente' AS source, id, scraping_date, upload_date, editor, title, content, article_url, created_at FROM jentestore_trends
UNION ALL
SELECT 'marieclaire' AS source, id, scraping_date, upload_date, editor, title, content, article_url, created_at FROM marieclaire_trends
UNION ALL
SELECT 'vogue' AS source, id, scraping_date, upload_date, editor, title, content, article_url, created_at FROM vogue_trends
UNION ALL
SELECT 'wkorea' AS source, id, scraping_date, upload_date, editor, title, content, article_url, created_at FROM wkorea_trends
UNION ALL
SELECT 'wwdkorea' AS source, id, scraping_date, upload_date, editor, title, content, article_url, created_at FROM wwdkorea_trends;
"""

# 실행
cursor.execute(create_view_sql)
conn.commit()

print("VIEW 테이블 생성 완료!")

# 연결 종료
cursor.close()
conn.close()


VIEW 테이블 생성 완료!


In [3]:
## Qdrant에 데이터 저장 ##
# 1. MySQL에서 데이터 가져와서 Qdrant에 벡터라이징 해서 저장

import pymysql
import hashlib
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

# 1️⃣ Qdrant 연결
qdrant = QdrantClient("http://localhost:6333")

# 2️⃣ 한국어 임베딩 모델 로드
model = SentenceTransformer("upskyy/bge-m3-korean")
vector_size = model.get_sentence_embedding_dimension()  # 벡터 크기 자동 확인

# 3️⃣ Qdrant 컬렉션 생성 (이미 존재하면 건너뜀)
collection_name = "fashion_trends"

if not qdrant.collection_exists(collection_name):
    qdrant.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
    )

# 4️⃣ MySQL 연결 (Unicode 지원 추가)
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="0805",
    db="fashion_trends",
    charset="utf8mb4",
    use_unicode=True
)
cursor = conn.cursor()

# 5️⃣ 단일 뷰(all_trends)에서 데이터 가져와서 Qdrant에 저장
cursor.execute("SELECT id, title, content, upload_date, article_url, source FROM all_trends")
rows = cursor.fetchall()

# 6️⃣ 데이터 벡터화 후 Qdrant에 저장
total_count = 0

for row in rows:
    doc_id, title, content, date, url, source = row

    # 🔹 content가 비어 있으면 건너뛰기
    if not content.strip():
        continue

    # 🔹 유니크 ID 생성 (source 테이블명 + id 해싱)
    unique_id = int(hashlib.md5(f"{source}_{doc_id}".encode()).hexdigest(), 16) % (10**9)

    # 🔹 임베딩 생성
    embedding = model.encode(content, convert_to_numpy=True).tolist()

    # 🔹 Qdrant에 저장
    qdrant.upsert(
        collection_name=collection_name,
        points=[
            PointStruct(
                id=unique_id,
                vector=embedding,
                payload={
                    "title": title,
                    "date": str(date),
                    "url": url,
                    "content": content,
                    "source": source
                }
            )
        ]
    )

    total_count += 1

print(f"🎉 총 {total_count}개 문서 벡터 저장 완료.")

# 7️⃣ MySQL 연결 종료
cursor.close()
conn.close()


c:\Users\Admin\miniconda3\envs\trends\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🎉 총 87개 문서 벡터 저장 완료.


In [4]:
import os
import qdrant_client
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from langgraph.graph import StateGraph, END
from langchain.schema import Document
from dataclasses import dataclass, field
from typing import List
from dotenv import load_dotenv

# 🔹 1. .env 파일에서 API 키 불러오기
load_dotenv()  
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")  

# 🔹 1-1. Google Gemini API 설정
genai.configure(api_key=GEMINI_API_KEY)  

# 🔹 2. QdrantDB 클라이언트 및 컬렉션 불러오기
qdrant = qdrant_client.QdrantClient("http://localhost:6333")
collection_name = "fashion_trends"

# 🔹 3. bge-m3-korean 임베딩 모델 로드
embedding_model = SentenceTransformer("upskyy/bge-m3-korean")

# 🔹 4. RAG 시스템의 상태 정의
@dataclass
class RAGState:
    query: str = ""
    documents: List[Document] = field(default_factory=list)
    response: str = ""  

# 🔹 5. 검색 노드 (Qdrant에서 유사 문서 검색)
def search_node(state: RAGState) -> RAGState:
    query_embedding = embedding_model.encode(state.query, convert_to_numpy=True).tolist()
    
    results = qdrant.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=3  # ✅ 상위 3개 문서 검색
    )

    docs = [
        Document(
            page_content=res.payload["title"] + "\n" + res.payload["url"],
            metadata={"date": res.payload["date"]}
        )
        for res in results
    ]

    return RAGState(query=state.query, documents=docs, response="")

# 🔹 6. LLM 응답 생성 노드 (Structured Report 텍스트 생성)
def generate_node(state: RAGState) -> RAGState:
    context = "\n\n".join([doc.page_content for doc in state.documents])

    prompt = f"""
    당신은 패션 바잉 MD를 위한 트렌드 분석 AI입니다.
    다음 기사 내용을 참고하여 최신의 패션 트렌드를 분석하고 Structured Output 형식의 보고서를 작성하세요.

    🔹 예제 보고서 형식:
    
    📊 {state.query} 보고서
    
    ✅ 트렌드 요약
    [트렌드 요약 내용]

    ✅ 주요 트렌드
    1. [트렌드 이름]: [트렌드 설명]
    2. [트렌드 이름]: [트렌드 설명]
    3. [트렌드 이름]: [트렌드 설명]

    ✅ 바잉 추천
    📌 구매해야 할 아이템:
    - [아이템 1]: 이유
    - [아이템 2]: 이유
    - [아이템 3]: 이유

    📌 피해야 할 아이템:
    - [아이템 1]: 이유
    - [아이템 2]: 이유

    ✅ 추가 인사이트
    - [시장 분석 인사이트]
    - [소비자 행동 트렌드]
    - 참고 URL: [기사 URL]

    🔹 기사 데이터:
    {context}

    🔹 위와 같은 형식으로 **Structured Output 형태의 보고서를 작성하세요.**
    """

    # ✅ Gemini API 호출 (올바른 방식 적용)
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(prompt)

    return RAGState(query=state.query, documents=state.documents, response=response.text)

# 🔹 7. LangGraph Workflow 생성
workflow = StateGraph(RAGState)
workflow.add_node("search", search_node)
workflow.add_node("generate", generate_node)
workflow.set_entry_point("search")
workflow.add_edge("search", "generate")
workflow.add_edge("generate", END)
app = workflow.compile()

# 🔹 8. 실행 함수 (Structured Report 반환)
def run_rag(query: str):
    state = RAGState(query=query)
    result = app.invoke(state)

    return result.get("response", "❌ 보고서 생성 실패!")

# ✅ 실행 예시
query = "2025년 봄 패션 바잉 전략을 위한 트렌드 분석"
report = run_rag(query)

# ✅ 최종 보고서 출력
print("\n📊 패션 바잉 전략 보고서:\n")
print(report)


C:\Users\Admin\AppData\Local\Temp\ipykernel_24160\544241702.py:36: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(



📊 패션 바잉 전략 보고서:

## 📊 2025년 봄 패션 바잉 전략을 위한 트렌드 분석 보고서

**✅ 트렌드 요약**

2025년 봄/여름 패션 트렌드는 실용성과 스타일을 겸비한 아이템들이 주를 이룰 것으로 예상됩니다. 특히 일자 바지와 어울리는 다양한 슈즈 스타일이 주목받고 있으며, 데님, 프린지, 뉴트럴 컬러, 보헤미안 스타일, 플라워 패턴, 스트라이프 패턴, 맥시 드레스, 그리고 짧은 스커트와 같은 핵심 트렌드가 부상하고 있습니다.

**✅ 주요 트렌드**

1.  **일자 바지 & 슈즈 스타일링:** 일자 바지는 꾸준한 인기를 유지하며, 다양한 슈즈와의 조합을 통해 스타일 변화를 줄 수 있습니다. (아래 바잉 추천 참고)
2.  **데님 (Denim):** 데님 소재는 2025년 봄/여름에도 주요 트렌드 중 하나로 자리 잡을 것으로 예상됩니다. 다양한 워싱, 톤, 디자인의 데님 아이템에 주목해야 합니다.
3.  **프린지 (Fringe):** 프린지 디테일은 의상에 움직임과 개성을 더하는 요소로 활용됩니다. 재킷, 스커트, 가방 등 다양한 아이템에서 프린지를 찾아볼 수 있습니다.
4.  **뉴트럴 컬러 (Neutral Colors):** 베이지, 아이보리, 카멜 등 뉴트럴 컬러는 시대를 초월하는 클래식함을 제공합니다. 톤온톤 스타일링을 통해 세련된 룩을 연출할 수 있습니다.
5.  **보헤미안 스타일 (Bohemian Style):** 자유로운 감성의 보헤미안 스타일은 플라워 패턴, 에스닉 프린트, 루즈한 실루엣 등으로 표현됩니다. 편안하면서도 스타일리시한 룩을 연출할 수 있습니다.
6.  **플라워 패턴 (Floral Patterns):** 봄을 대표하는 플라워 패턴은 다양한 크기와 색상으로 표현됩니다. 드레스, 블라우스, 스커트 등 다양한 아이템에서 찾아볼 수 있습니다.
7.  **스트라이프 패턴 (Stripe Patterns):** 클래식한 스트라이프 패턴은 다양한 굵기와 방향으로 표현됩니다. 캐주얼하면서도 세련된 룩을 연출할 수 있습니다.
